In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from pprint import pprint

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.L3_D3_db
collection = db.list_of_lists_of_lists

In [4]:
# URL of page to be scraped
url = 'https://en.wikipedia.org/wiki/List_of_lists_of_lists'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [192]:
# Initialize post dictionary with root node
root = {"name":soup.find('h1').text,
              "children": []
        }

current_h1 = root["children"]

In [198]:
# all_siblings: ResultSet of all generations after Root Node
all_siblings = soup.find('div', class_='mw-parser-output').find_all(['h2','h3','h4','ul'], recursive=False)

def remove_edit(string):
    if "[edit]" in string:
        string = string[:-6]
    return string

In [233]:
all_siblings[4].find('ul').find('li').find('a').get('href')

'/wiki/Lists_of_100_best_books'

In [ ]:
# Function for processing nested ul's / li's. Takes ResultSet of li's as arg
def process_ul(li_results):
    
    # Iterate through each li
    for j, li in enumerate(li_results):
        
        # Always add each 
        ul_parent.append({'name':li.text, 
                          'children':[]
                         })

        ul_parent = ul_parent[j]['children']

        possible_ul = li_results[j].find('ul')

        if possible_ul:
            more_li = possible_ul.find_all('li')
            process_ul(more_li)
        else:
            continue
'''    
TO DO:
RECURSION!
if [href] (exists): .append with 'link'

base case: last li of resultset does not contain ul?
    do not process_ul! should be done here bc entries are added as we go? loop finishes with last li

if ul:
    process_ul (find_all(li) resultset - contains only li!)
        for each resulting li:
            append .text to ul_parent
            if result.find('ul') (li has ul):
                set new ul_parent***???
                process_ul (find_all(li) resultset)
            else (li doesn't have ul):
'''

In [213]:
# Initialize counters for h2, h3, h4 levels
h2_count = 0
h3_count = 0
h4_count = 0

# Need enumerate to save index
for i, sibling in enumerate(all_siblings):
    
    # Check if this node/sibling contains a link.
    # If yes, save href as _link_ to be added to final object. If no, assign None to _link_
    has_link = sibling.find('a')
    if has_link:
        link = has_link.get('href')
    else:
        link = None
    
    if sibling.name == 'ul':
        # Generate ResultSet of all li's under this ul. Recursively process all ul/li's
        li_results = all_siblings[i].find_all('li')
        process_ul(li_results)
    
    # Remove "[edit]" text from the end of header strings
    list_name = remove_edit(sibling.text)
    
    elif sibling.name == 'h2':
        # Reset h3 count, since new h2
        h3_count = 0
        
        # Append as new child to current/active h1 parent. 
        # Add name (.text), empty list of children (even if won't be populated), and link (href) if exists!
        current_h1.append({'name':list_name, 
                           'children':[]
                          })
        
        # Set current_h2 to THIS level's children, reset on every h2 encounter
        current_h2 = current_h1[h2_count]['children']
        
        # Set this h2 as possible next ul_parent (in case ul comes next)
        ul_parent = current_h2
        
        # Increase counter. This ensures the next h3 is added as a child to this h2
        h2_count += 1
        
    elif sibling.name == 'h3':  
        
        # Same as code for h2
        h4_count = 0
        current_h2.append({'name':list_name, 
                           'children':[]
                          })
        current_h3 = current_h2[h3_count]['children']
        ul_parent = current_h3
        h3_count += 1
    
    elif sibling.name == 'h4':
        
        # Same as code for h2
        current_h3.append({'name':list_name, 
                           'children':[]
                          })
        current_h4 = current_h3[h4_count]['children']
        ul_parent = current_h4
        h4_count += 1

In [214]:
root

{'name': 'List of lists of lists',
 'children': [{'name': 'General reference',
   'children': [{'name': 'Lists of academic journals', 'children': []},
    {'name': 'Lists of important publications in science', 'children': []},
    {'name': 'Lists of unsolved problems', 'children': []}]},
  {'name': 'Culture and the arts',
   'children': [{'name': 'Literature',
     'children': [{'name': 'Lists of books\nLists of 100 best books\nLists of banned books\nLists of The New York Times Fiction Best Sellers\nLists of The New York Times Non-Fiction Best Sellers\nPublishers Weekly lists of bestselling novels in the United States',
       'children': []},
      {'name': 'Lists of 100 best books', 'children': []},
      {'name': 'Lists of banned books', 'children': []},
      {'name': 'Lists of The New York Times Fiction Best Sellers',
       'children': []},
      {'name': 'Lists of The New York Times Non-Fiction Best Sellers',
       'children': []},
      {'name': 'Publishers Weekly lists of bes